In [1]:
from typing import Dict, List, Tuple

import arviz as az
import numpy as np
import pandas as pd
import xarray as xr
from bokeh.layouts import column
from bokeh.models import (
    Band,
    ColumnDataSource,
    DatetimeTickFormatter,
    GeoJSONDataSource,
    HoverTool,
    LabelSet,
    NumeralTickFormatter,
    Select,
    Span,
    Title,
)
from bokeh.palettes import cividis, inferno, viridis
from bokeh.plotting import figure, output_file, output_notebook, show
from scipy.special import expit as logistic

In [2]:
# output_notebook()

In [3]:
def standardize(series):
    """Standardize a pandas series"""
    return (series - series.mean()) / series.std()


def dates_to_idx(timelist):
    """Convert datetimes to numbers in reference to a given date. Useful for posterior predictions."""

    reference_time = timelist[0]
    t = (timelist - reference_time) / np.timedelta64(1, "M")

    return np.asarray(t)

In [4]:

df = pd.read_csv("../data/complete_popularity_data.csv", index_col=0, parse_dates=True)

PREDICTION_COORDS = pd.read_csv(
    "../data/projecoes/cambio_prediction_coords.csv", index_col=0, parse_dates=["temporal"]
)

raw_polls = pd.read_csv("../data/raw_polls.csv", index_col=0, parse_dates=True)

trace_predictions = az.from_netcdf("../data/projecoes/cambio_trace_predicoes.nc")
trace_raw_fundamental = az.from_netcdf("../data/projecoes/cambio_trace_bruto.nc")
pp_prop_atual = xr.open_dataset("../data/projecoes/cambio_post_pred_pos.nc")
pp_prop_baixo = xr.open_dataset("../data/projecoes/cambio_post_pred_pos_04.nc")
pp_prop_alto = xr.open_dataset("../data/projecoes/cambio_post_pred_pos_07.nc")

df

,mensal,presidente,positiva,regular,negativa,amostra,erro,nsnr,positiva_int,negativa_int,...,cambio_lag,ipca_mes,ipca_var_mensal,ipca_var_anual,Inflacao_acumulada,pib_var_mensal,pib_var_anual,ipca_no_ano,pib_no_ano,cambio_defl
mes,,,,,,,,,,,,,,,,,,,,,
1995-01-31,1995-01,Fernando Henrique I,0.570000,0.230000,0.090000,1000.000000,3.000000,10.000000,570,90,...,0.85,17.28,1.70,631.54,277.54,6.37,930.70,1.70,6.37,0.819076
1995-02-28,1995-02,Fernando Henrique I,0.570000,0.230000,0.090000,1500.000000,2.500000,10.000000,854,135,...,0.85,10.54,1.02,426.83,278.56,6.22,762.33,2.72,12.59,0.821422
1995-03-31,1995-03,Fernando Henrique I,0.415000,0.407500,0.125000,1300.000000,2.500000,5.250000,539,162,...,0.84,16.19,1.55,274.78,280.11,20.16,604.48,4.27,32.75,0.861152
1995-04-30,1995-04,Fernando Henrique I,0.390000,0.400000,0.170000,2000.000000,3.000000,3.000000,780,340,...,0.89,25.77,2.43,169.05,282.54,-3.72,350.47,6.70,29.03,0.864688
1995-05-31,1995-05,Fernando Henrique I,0.380000,0.425000,0.155000,1600.000000,3.000000,3.500000,608,248,...,0.91,29.00,2.67,91.79,285.21,-2.49,174.82,9.37,26.54,0.850589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-31,2021-05,Jair Bolsonaro,0.268889,0.218111,0.493444,2089.666667,2.266667,1.787500,561,1031,...,5.56,47.25,0.83,8.06,449.76,-1.05,22.65,3.18,1.51,5.238223
2021-06-30,2021-06,Jair Bolsonaro,0.271213,0.208163,0.501875,1936.750000,2.462500,1.950000,525,972,...,5.29,30.42,0.53,8.35,450.29,0.63,17.17,3.71,2.14,4.998209
2021-07-31,2021-07,Jair Bolsonaro,0.254833,0.206333,0.521333,1926.500000,2.508333,1.733333,490,1004,...,5.03,55.39,0.96,8.99,451.25,4.00,14.45,4.67,6.14,5.101426


In [5]:
time = dates_to_idx(df.index)
time[:10]

array([0.        , 0.91993675, 1.93843816, 2.92408468, 3.94258609,
       4.92823261, 5.94673402, 6.96523543, 7.95088195, 8.96938335])

In [6]:
# change-points / Linzer model
# uncertainty in y (pollsters weights)
# economy: GRW

In [7]:
def get_data_source(
    trace: az.InferenceData, post_pred_samples: xr.DataArray) -> pd.DataFrame:
    source_df = (
        post_pred_samples.stack(sample=("chain", "draw"))
        .to_pandas()
        .droplevel(0, axis=1)
    )
    
    source_df.columns = source_df.columns.astype(str)

    source_df["baseline"] = logistic(trace.predictions["baseline"]).mean().data
    source_df["baseline_lower"] = (
        logistic(az.hdi(trace.predictions)["baseline"]).sel(hdi="lower").data
    )
    source_df["baseline_upper"] = (
        logistic(az.hdi(trace.predictions)["baseline"]).sel(hdi="higher").data
    )

    source_df["median_app"] = post_pred_samples.median(dim=("chain", "draw")).data
    source_df["median_low"] = np.squeeze(
        az.hdi(post_pred_samples, hdi_prob=0.75).sel(hdi="lower").to_array().data
    )
    source_df["median_high"] = np.squeeze(
        az.hdi(post_pred_samples, hdi_prob=0.75).sel(hdi="higher").to_array().data
    )

    return source_df

In [8]:
def samples_subset(data_source: pd.DataFrame, frac: float = 0.1) -> Dict[str, List]:

    sub_source = data_source.filter(regex="\d", axis="columns").sample(
        frac=frac, replace=True, axis="columns"
    )

    dates = []
    draws = []
    for draw in sub_source.columns:
        dates.append(sub_source.index.values)
        draws.append(sub_source[draw].values)

    return {"dates": dates, "draws": draws}

In [9]:
source_annotations = ColumnDataSource(
    data=dict(
        dates=[
            pd.to_datetime("1995-01-01"),
            pd.to_datetime("1999-01-01"),
            pd.to_datetime("2003-01-01"),
            pd.to_datetime("2007-01-01"),
            pd.to_datetime("2011-01-01"),
            pd.to_datetime("2015-01-01"),
            pd.to_datetime("2019-01-01"),
            pd.to_datetime("2009-05-01"),
        ],
        ys=[0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.36],
        events=[
            "FHC I",
            "FHC II",
            "Lula I",
            "Lula II",
            "Dilma I",
            "Dilma/Temer",
             "Bolsonaro",
            "Média histórica",
        ],
    )
);


In [10]:
def make_plot(
    subtitle: str,
    palette,
    random_draws: Dict[str, List],
    data_source: pd.DataFrame,
    post_pred_samples: xr.Dataset,
):
    CDS = ColumnDataSource(data_source)

    p = figure(
        plot_width=1200,
        plot_height=425,
        sizing_mode="scale_both",
        background_fill_color="#f2f2f2",
        border_fill_color="#f2f2f2",
        x_axis_type="datetime",
        title="Popularidade dos Presidentes",
        x_range=(
            pd.to_datetime("1994-08-01"),
            PREDICTION_COORDS["temporal"].iloc[-1] + pd.DateOffset(months=6),
        ),
        y_range=(0, 1),
        toolbar_location="right",
        tools="xpan, box_zoom, xwheel_zoom, crosshair, reset, save",
    )
    p.xaxis[0].formatter = DatetimeTickFormatter(months="%b %Y", days="%d/%m")
    p.yaxis[0].formatter = NumeralTickFormatter(format="00%")
    p.add_layout(
        Title(
            text=f"1 trimestre de projeção, se o dólar {subtitle}",
            align="center",
            text_font_style="italic",
            text_font_size="1.2em",
        ),
        "above",
    )
    p.title.text_font_size = "1.6em"
    p.title.align = "center"
    p.grid.grid_line_alpha = 0.5
    p.xaxis.axis_label = "Período"
    p.yaxis.axis_label = "% Popularidade"

    p.multi_line(
        xs=random_draws["dates"],
        ys=random_draws["draws"],
        color=palette[4],
        legend_label="Random samples",
    )
    p.patch(
        np.concatenate((data_source.index.values, data_source.index.values[::-1])),
        np.concatenate(
            (
                np.squeeze(az.hdi(post_pred_samples).sel(hdi="lower").to_array()),
                np.squeeze(az.hdi(post_pred_samples).sel(hdi="higher").to_array())[
                    ::-1
                ],
            )
        ),
        color=palette[3],
        line_alpha=0.4,
        fill_alpha=0.4,
        legend_label="95% HDI",
    )
    hdi = p.patch(
        np.concatenate((data_source.index.values, data_source.index.values[::-1])),
        np.concatenate(
            (
                np.squeeze(
                    az.hdi(post_pred_samples, hdi_prob=0.75).sel(hdi="lower").to_array()
                ),
                np.squeeze(
                    az.hdi(post_pred_samples, hdi_prob=0.75)
                    .sel(hdi="higher")
                    .to_array()
                )[::-1],
            )
        ),
        color=palette[2],
        line_alpha=0,
        fill_alpha=0.5,
        legend_label="75% HDI",
    )
    p.patch(
        np.concatenate((data_source.index.values, data_source.index.values[::-1])),
        np.concatenate(
            (
                np.squeeze(
                    az.hdi(post_pred_samples, hdi_prob=0.5).sel(hdi="lower").to_array()
                ),
                np.squeeze(
                    az.hdi(post_pred_samples, hdi_prob=0.5).sel(hdi="higher").to_array()
                )[::-1],
            )
        ),
        color=palette[1],
        line_alpha=0,
        fill_alpha=0.6,
        legend_label="50% HDI",
    )
    median_line = p.line(
        "temporal",
        "median_app",
        color=palette[0],
        line_width=2,
        legend_label="Median",
        source=CDS,
    )
    p.scatter(
        raw_polls.index.values,
        raw_polls.positiva.values,
        size=6,
        color=palette[5],
        legend_label="Pesquisas observadas",
        alpha=0.7,
    )

    labels = LabelSet(
        x="dates",
        y="ys",
        text="events",
        level="glyph",
        text_color="gray",
        text_font_style="italic",
        text_font_size="1em",
        text_align="center",
        source=source_annotations,
    )
    vline_0 = Span(
        location=source_annotations.data["dates"][0],
        dimension="height",
        line_color="gray",
        line_dash="dotted",
        line_width=1.2,
    )
    vline_1 = Span(
        location=source_annotations.data["dates"][1],
        dimension="height",
        line_color="gray",
        line_dash="dotted",
        line_width=1.2,
    )
    vline_2 = Span(
        location=source_annotations.data["dates"][2],
        dimension="height",
        line_color="gray",
        line_dash="dotted",
        line_width=1.2,
    )
    vline_3 = Span(
        location=source_annotations.data["dates"][3],
        dimension="height",
        line_color="gray",
        line_dash="dotted",
        line_width=1.2,
    )
    vline_4 = Span(
        location=source_annotations.data["dates"][4],
        dimension="height",
        line_color="gray",
        line_dash="dotted",
        line_width=1.2,
    )

    vline_5 = Span(
        location=source_annotations.data["dates"][5],
        dimension="height",
        line_color="gray",
        line_dash="dotted",
        line_width=1.2,
    )
    
    
    vline_6 = Span(
        location=source_annotations.data["dates"][6],
        dimension="height",
        line_color="gray",
        line_dash="dotted",
        line_width=1.2,
    )

    fifty_line = Span(
        location=0.5,
        dimension="width",
        line_color="gray",
        line_dash="dashed",
        line_width=1.2,
    )
    hist_band = Band(
        base="temporal",
        lower="baseline_lower",
        upper="baseline_upper",
        source=CDS,
        fill_color="gray",
        fill_alpha=0.2,
    )
    hist_avg_line = Span(
        location=CDS.data["baseline"][0],
        dimension="width",
        line_color="gray",
        line_dash="dashdot",
        line_width=2,
    )

    p.renderers.extend(
        [
            labels,
            vline_0,
            vline_1,
            vline_2,
            vline_3,
            vline_4,
            vline_5,
            vline_6,
            fifty_line,
            hist_band,
            hist_avg_line,
        ]
    )

    p.legend.click_policy = "hide"
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"
    p.legend.background_fill_color = "#f2f2f2"
    p.legend.background_fill_alpha = 0.6

    # Add the HoverTool to the figure
    TOOLTIPS = [
        ("Mediana", "@median_app{00%} e @temporal{%b %Y}"),
        ("75% chance entre", "@median_low{00%} e @median_high{00%}"),
        ("Média histórica entre", "@baseline_lower{00%} e @baseline_upper{00%}"),
    ]
    p.add_tools(
        HoverTool(
            tooltips=TOOLTIPS,
            formatters={"@temporal": "datetime"},
            mode="vline",
            renderers=[median_line],
        )
    )

    return p

In [11]:
pp_prop_atual

<xarray.Dataset>
Dimensions:        (chain: 1, draw: 1000, temporal: 324)
Coordinates:
  * chain          (chain) int64 0
  * draw           (draw) int64 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
  * temporal       (temporal) datetime64[ns] 1995-01-31 ... 2021-12-31
Data variables:
    post_pred_pos  (chain, draw, temporal) float64 ...

In [12]:
source_df1 = get_data_source(trace_raw_fundamental, pp_prop_atual["post_pred_pos"])
source_df2 = get_data_source(trace_raw_fundamental, pp_prop_baixo["post_pred_pos_baixo"])
source_df3 = get_data_source(trace_raw_fundamental, pp_prop_alto["post_pred_pos_alto"])

In [13]:
random_draws1 = samples_subset(source_df1)
random_draws2 = samples_subset(source_df2)
random_draws3 = samples_subset(source_df3)

In [14]:
p1 = make_plot(
    subtitle=f"permanecer em R$ {df.cambio_defl.iloc[-1].round(2)}",
    palette=cividis(6),
    random_draws=random_draws1,
    data_source=source_df1,
    post_pred_samples=pp_prop_atual,
)
p2 = make_plot(
    subtitle="cair para R$ 4.00",
    palette=viridis(6),
    random_draws=random_draws2,
    data_source=source_df2,
    post_pred_samples=pp_prop_baixo,
)
p3 = make_plot(
    subtitle="subir para R$ 7.00",
    palette=inferno(6),
    random_draws=random_draws3,
    data_source=source_df3,
    post_pred_samples=pp_prop_alto,
)

p2.title.text = ''
p3.title.text = ''
p2.x_range = p1.x_range
p3.x_range = p1.x_range

show(column(p1, p2, p3))

In [15]:
%load_ext watermark
%watermark -a @dmarcelinobr -n -u -v -iv

Author: @dmarcelinobr

Last updated: Tue Oct 05 2021

Python implementation: CPython
Python version       : 3.9.6
IPython version      : 7.26.0

xarray: 0.19.0
numpy : 1.21.1
arviz : 0.11.2
pandas: 1.3.1

